<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_09_3_dalle_existing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google CoLab Instructions
The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    drive.mount('/content/drive')
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain openai langchain_openai
    !pip install --upgrade langchain openai
    !pip install schedule

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Note: using Google CoLab


In [ ]:
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from openai import OpenAI
import uuid
import time
import schedule

# Schedule tasks
generateTwentyImages = False #or True  # Condition to start generating images

client = OpenAI()

# Function to download and load an image from a URL
def load_image_from_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Function to load the image
def load_image_from_url2(image_path):
    img = Image.open(image_path)  # Open the image from the given path
    return img

def generate_images() :
  print("Generating an image...")
  # URLs for the images
  image_url_1 = "https://peterstroessler.com/src/images/hero/hero-image.png"
  image_url_2 = "/content/drive/My Drive/jeff_heaton/hero-image-free.png"

  # Load the images from URLs
  image_1 = load_image_from_url(image_url_1)
  image_2 = load_image_from_url2(image_url_2)

  # Save the images to in-memory files
  img_byte_array_1 = BytesIO()
  img_byte_array_2 = BytesIO()
  image_1.save(img_byte_array_1, format='PNG')
  image_2.save(img_byte_array_2, format='PNG')

  # Reset the buffer's position to the start
  img_byte_array_1.seek(0)
  img_byte_array_2.seek(0)

  # Use the OpenAI API to edit the images with in-memory files
  response = client.images.edit(
    image=img_byte_array_1,
    mask=img_byte_array_2,
    prompt="blue color shirt",
    n=1,
    size="1024x1024"
  )

  # Access the first image URL from the response
  edited_image_url = response.data[0].url

  # Download the edited image from the URL
  edited_image_response = requests.get(edited_image_url)
  resulting_image = Image.open(BytesIO(edited_image_response.content))
  if not generateTwentyImages == True:
    # Display the resulting image using matplotlib
    plt.imshow(resulting_image)
    plt.axis('off')  # Hide the axes
    plt.show()
  generated_uuid = uuid.uuid4()
  new_image_filename = f"/content/drive/My Drive/jeff_heaton/images/image_{generated_uuid}.png"
  resulting_image.save(new_image_filename, "PNG")
  return new_image_filename
import schedule
import time

# Variables to track total image count and calls per minute
total_images_generated = 0
call_count = 0
'''
def generate_images():
    """Simulates image generation."""
    print("Generating an image...")
'''
def func():
    """
    Function scheduled to run every minute to generate up to 3 images
    until the total reaches 20.
    """
    global total_images_generated, call_count

    # Check if the total number of images has reached 20
    if total_images_generated >= 20:
        print("Completed generating 20 images. Stopping scheduler.")
        return schedule.clear()  # Stop all scheduled jobs

    # Generate images up to 3 times per minute
    if call_count < 3:
        generate_images()
        total_images_generated += 1
        call_count += 1
    else:
        print("Max calls reached for this minute.")

def reset_counter():
    """Resets the per-minute call counter."""
    global call_count
    call_count = 0
    print("Counter reset for the new minute.")

if generateTwentyImages:
    print("Starting image generation...")
    schedule.every(1).minutes.do(reset_counter)  # Reset counter every minute
    schedule.every(20).seconds.do(func)  # Call func every 20 seconds (up to 3 per minute)

    try:
        while True:
            schedule.run_pending()
            time.sleep(1)  # Prevent excessive CPU usage
    except KeyboardInterrupt:
        print("Scheduler stopped by user.")
else :
  print("generating single image")
  new_image_filename = generate_images()

Starting image generation...
Generating an image...
Generating an image...
I'm working...
Generating an image...
Generating an image...
Generating an image...
Counter reset for the new minute.
Generating an image...
Counter reset for the new minute.
Counter reset for the new minute.
Generating an image...
Generating an image...
Generating an image...
Max calls reached for this minute.
Counter reset for the new minute.
Counter reset for the new minute.
Generating an image...
I'm working...
Generating an image...
Scheduler stopped by user.


In [ ]:
from google.colab import files
if not generateTwentyImages :
  files.download(new_image_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>